<a href="https://colab.research.google.com/github/ponakilan/vid-anomaly/blob/main/RC_IPAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! 7z x '/content/drive/MyDrive/Research Credit/IPAD_dataset.zip'


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Research Credit/                                                 1 file, 8920232221 bytes (8507 MiB)

Extracting archive: /content/drive/MyDrive/Research Credit/IPAD_dataset.zip
  0% 4096 Open               10% 61440 Open                10% 65536 Open                11% 69632 Open                29% 176128 Open                 54% 327680 Open                 81% 487424 Open                --
Path = /content/drive/MyDrive

In [ ]:
import tensorflow as tf

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory="/content/drive/My Drive/Research Credit/IPAD_Dataset"
)